In [ ]:
import pandas as pd
import http
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

!pip install scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, recall_score ,precision_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [ ]:
data = pd.read_csv('/content/clean_data newest newest newest.csv')
print(data.head())

   Sleep  Appetite  Interest  Fatigue  Worthlessness  Concentration  \
0    1.0       1.0       1.0      5.0            5.0            1.0   
1    2.0       5.0       5.0      1.0            1.0            5.0   
2    5.0       2.0       2.0      2.0            2.0            2.0   
3    1.0       1.0       1.0      5.0            5.0            1.0   
4    2.0       5.0       5.0      1.0            1.0            5.0   

   Agitation  Suicidal Ideation  Sleep Disturbance  Aggression  Panic Attacks  \
0        5.0                5.0                1.0         5.0            5.0   
1        1.0                1.0                5.0         1.0            1.0   
2        2.0                2.0                2.0         2.0            2.0   
3        5.0                5.0                1.0         5.0            5.0   
4        1.0                1.0                5.0         1.0            1.0   

   Hopelessness  Restlessness  Low Energy  Depression State  
0           5.0         

In [ ]:
print(data['Depression State'].unique())

[1. 2. 3. 0.]


In [ ]:
print(data)

     Sleep  Appetite  Interest  Fatigue  Worthlessness  Concentration  \
0      1.0       1.0       1.0      5.0            5.0            1.0   
1      2.0       5.0       5.0      1.0            1.0            5.0   
2      5.0       2.0       2.0      2.0            2.0            2.0   
3      1.0       1.0       1.0      5.0            5.0            1.0   
4      2.0       5.0       5.0      1.0            1.0            5.0   
..     ...       ...       ...      ...            ...            ...   
808    5.0       5.0       5.0      5.0            5.0            5.0   
809    5.0       5.0       5.0      5.0            5.0            5.0   
810    5.0       5.0       5.0      5.0            5.0            5.0   
811    5.0       5.0       5.0      5.0            5.0            5.0   
812    5.0       5.0       5.0      5.0            5.0            5.0   

     Agitation  Suicidal Ideation  Sleep Disturbance  Aggression  \
0          5.0                5.0                1.0   

In [ ]:
data = pd.DataFrame(data)

X = np.array(data.drop(['Depression State'], axis=1))
y = np.array(data['Depression State'])

data = pd.concat([pd.DataFrame(X), pd.Series(y, name='Depression State')], axis=1)
data_copy = data.dropna()

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y, random_state=42, test_size = 0.25)

In [ ]:
print('Samples in Train Set:', len(X_train))
print('Samples in Test Set:', len(X_test))

Samples in Train Set: 609
Samples in Test Set: 204


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
print(data)

       0    1    2    3    4    5    6    7    8    9   10   11   12   13  \
0    1.0  1.0  1.0  5.0  5.0  1.0  5.0  5.0  1.0  5.0  5.0  5.0  5.0  5.0   
1    2.0  5.0  5.0  1.0  1.0  5.0  1.0  1.0  5.0  1.0  1.0  1.0  1.0  1.0   
2    5.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0   
3    1.0  1.0  1.0  5.0  5.0  1.0  5.0  5.0  1.0  5.0  5.0  5.0  5.0  5.0   
4    2.0  5.0  5.0  1.0  1.0  5.0  1.0  1.0  5.0  1.0  1.0  1.0  1.0  1.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
808  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0   
809  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0   
810  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0   
811  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0   
812  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0   

     Depression State  
0                 1.0  
1                 2.0  
2  

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=8)

X_bal, y_bal = sm.fit_resample(X_train, y_train)



# Neural Network

In [ ]:
!pip install keras-tuner --upgrade
import tensorflow as tf
from kerastuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam

## Regularization

In [ ]:
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(keras.Input(shape=(14,)))
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Hyperparameter Tunning

In [ ]:

def build_model(hp):
    model = models.Sequential()
    model.add(keras.Input(shape=(14,)))
    model.add(layers.Dense(hp.Int('units', min_value=64, max_value=256, step=64),
                           activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout', min_value=0.3, max_value=0.5, step=0.1)))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.01, 0.001, 0.005, 0.0001, 0.0005])),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def build_model(hp):
    model = models.Sequential()
    model.add(keras.Input(shape=(14,)))
    model.add(layers.Dense(hp.Int('units_1', min_value=128, max_value=512, step=64), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.4, step=0.1)))
    model.add(layers.Dense(hp.Int('units_2', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.3, step=0.1)))
    model.add(layers.Dense(hp.Int('units_3', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout_3', min_value=0.2, max_value=0.3, step=0.1)))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.01, 0.001, 0.0005])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    return model

In [ ]:
import shutil
shutil.rmtree('./untitled_project')

In [ ]:
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5)
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 16s]
val_accuracy: 0.6372548937797546

Best val_accuracy So Far: 0.6372548937797546
Total elapsed time: 00h 01m 35s


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
units_1 = best_hps.get('units_1')
units_2 = best_hps.get('units_2')
units_3 = best_hps.get('units_3')
dropout_rate_1 = best_hps.get('dropout_1')
dropout_rate_2 = best_hps.get('dropout_2')
dropout_rate_3 = best_hps.get('dropout_3')
learning_rate = best_hps.get('learning_rate')

In [ ]:
print(f"Optimal Units 1: {best_hps.get('units_1')}")
print(f"Optimal Units 2: {best_hps.get('units_2')}")
print(f"Optimal Units 3: {best_hps.get('units_3')}")
print(f"Optimal dropout Rate 1: {best_hps.get('dropout_1')}")
print(f"Optimal dropout Rate 2: {best_hps.get('dropout_2')}")
print(f"Optimal dropout Rate 3: {best_hps.get('dropout_3')}")
print(f"Optimal Learning Rate: {best_hps.get('learning_rate')}")

Optimal Units 1: 320
Optimal Units 2: 64
Optimal Units 3: 128
Optimal dropout Rate 1: 0.2
Optimal dropout Rate 2: 0.2
Optimal dropout Rate 3: 0.2
Optimal Learning Rate: 0.0005


## Define EarlyStopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=5,
    restore_best_weights=True,
)

## Define the model with Best Parameter Tunning

In [ ]:
final_model = models.Sequential()
final_model.add(keras.Input(shape=(14,)))
final_model.add(layers.Dense(units_1, activation='relu'))
final_model.add(layers.BatchNormalization())
final_model.add(layers.Dropout(dropout_rate_1))
final_model.add(layers.Dense(units_2, activation='relu'))
final_model.add(layers.BatchNormalization())
final_model.add(layers.Dropout(dropout_rate_2))
final_model.add(layers.Dense(units_3, activation='relu'))
final_model.add(layers.BatchNormalization())
final_model.add(layers.Dropout(dropout_rate_3))
final_model.add(layers.Dense(4, activation='softmax'))

final_model.compile(optimizer=Adam(learning_rate=learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

## Train the model

In [ ]:
history = final_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=500,
    batch_size=32,
    callbacks=[early_stopping]
)


Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.3363 - loss: 1.7617 - val_accuracy: 0.5490 - val_loss: 1.1083
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5328 - loss: 1.4594 - val_accuracy: 0.5294 - val_loss: 1.0692
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5260 - loss: 1.2698 - val_accuracy: 0.5343 - val_loss: 1.0444
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5321 - loss: 1.2588 - val_accuracy: 0.5294 - val_loss: 1.0320
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5355 - loss: 1.2557 - val_accuracy: 0.5294 - val_loss: 1.0172
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5390 - loss: 1.2139 - val_accuracy: 0.5490 - val_loss: 1.0137
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5595 - loss: 1.1950 - val_accuracy: 0.5343 - val_loss: 1.0111
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6237 - loss: 1.0648 - val_accuracy: 0.

## Evaluate the Model

In [ ]:
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f"Test Accuracy: {accuracy:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6204 - loss: 0.7802 
Test Loss: 0.8202
Test Accuracy: 0.60


In [ ]:
loss, accuracy = final_model.evaluate(X_train, y_train)
print(f'Train Loss: {loss:.4f}')
print(f"Train Accuracy: {accuracy:.2f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6545 - loss: 0.7829
Train Loss: 0.8029
Train Accuracy: 0.65
